In [1]:
import numpy as np
from functools import lru_cache

P = np.array([
    #  1    2    3    4    5    6    7    8    9   10   11 
    [.06, .50, .00, .00, .44, .00, .00, .00, .00, .00, .00], #1
    [.00, .10, .90, .00, .00, .00, .00, .00, .00, .00, .00], #2
    [.29, .00, .40, .00, .00, .02, .29, .00, .00, .00, .00], #3
    [.24, .18, .00, .05, .00, .00, .53, .00, .00, .00, .00], #4
    [.18, .44, .00, .00, .27, .00, .00, .11, .00, .00, .00], #5
    [.00, .00, .26, .07, .00, .09, .28, .00, .05, .00, .25], #6
    [.00, .00, .01, .17, .00, .16, .31, .00, .12, .09, .14], #7
    [.00, .00, .53, .00, .00, .00, .00, .47, .00, .00, .00], #8
    [.00, .00, .00, .00, .00, .23, .00, .16, .24, .37, .00], #9
    [.00, .00, .00, .00, .00, .00, .42, .00, .00, .58, .00], #10
    [.00, .00, .00, .00, .00, .00, .31, .62, .00, .00, .07], #11
    #  1    2    3    4    5    6    7    8    9   10   11
])

In [14]:
# 1. Вероятность того, что за 10 шагов система перейдет из состояния 1 в состояние 5

@lru_cache
def p_on_step_k(k):
    res_p = P.copy()
    for _ in range(k-1):
        res_p = res_p @ P
    return res_p

print(f'Вероятность перехода из 1 в 5 за 10 шагов: {p_on_step_k(10)[0][4]:.2f}')

Вероятность перехода из 1 в 5 за 10 шагов: 0.06


In [15]:
# 2. Вероятности состояний системы спустя 7 шагов
A0 = [.05, .17, .0, .19, .05, .02, .02, .17, .17, .14, .02]

def a_on_step_k(a, k):
    return a @ p_on_step_k(k)

res = a_on_step_k(A0, 7)
print('Вероятности состояний системы спустя 7 шагов:')
for i, item in enumerate(res):
    print(f'{i:>2}: {item}')

Вероятности состояний системы спустя 7 шагов:
 0: 0.0871546822271336
 1: 0.08129063264853481
 2: 0.21662844200142592
 3: 0.043295285922233404
 4: 0.05147282604516511
 5: 0.05317687583170191
 6: 0.21938228485790617
 7: 0.0780214109642064
 8: 0.0383312274968078
 9: 0.0837570454834561
10: 0.0474892865214288


In [4]:
# 3. Вероятность первого перехода за 9 шагов из состояния 7 в состояние 11

@lru_cache
def first_on_step_k(k):
    if k == 0:
        return np.eye(len(P))
    if k == 1:
        return P
    
    m = len(P)    
    res = np.zeros((m, m))
    
    pre = first_on_step_k(k-1)
    
    for i in range(m):
        for j in range(m):
            tmp = 0
            for k in range(m):
                tmp += (k != j) and (P[i][k] * pre[k][j])
            res[i, j] = tmp
    
    return res    

print(f'Вероятность первого перехода за 9 шагов из состояния 7 в состояние 11: \
{first_on_step_k(9)[6][10]}')

Вероятность первого перехода за 9 шагов из состояния 7 в состояние 11: 0.027672697647903262


In [5]:
# 4. Bероятность перехода из состояния 5 в состояние 10 не позднее чем за 10 шагов

def no_later_than_k(k, a, b):
    res = 0
    for i in range(1, k):
        res = res + first_on_step_k(i)[a-1][b-1]
    return res

print(f'Bероятность перехода из состояния 5 в состояние 10 не позднее чем за 10 шагов: \
{no_later_than_k(10, 5, 10)}')

Bероятность перехода из состояния 5 в состояние 10 не позднее чем за 10 шагов: 0.1379495907811075


In [6]:
# 5. Cреднее количество шагов для перехода из состояния 10 в состояние 3
def avg_steps(a, b):
    res = 0
    for i in range(10000):
        res += i * first_on_step_k(i)[a-1][b-1]
    return res

print(f'Cреднее количество шагов для перехода из состояния 10 в состояние 3: \
{avg_steps(10, 3)}')

Cреднее количество шагов для перехода из состояния 10 в состояние 3: 11.007828336481412


In [7]:
# 6. Bероятность первого возвращения в состояние 6 за 7 шагов

@lru_cache
def first_back_on_k(k, a):
    
    if k == 1:
        return P[a][a]
    
    return p_on_step_k(k)[a][a] -\
        sum([first_back_on_k(m, a) * p_on_step_k(k-m)[a][a] for m in range(1, k)])

print(f'Bероятность первого возвращения в состояние 6 за 7 шагов: \
{first_back_on_k(7, 5)}')

Bероятность первого возвращения в состояние 6 за 7 шагов: 0.0331950630166


In [8]:
# 7. Bероятность возвращения в состояние 11 не позднее чем за 6 шагов

def back_no_later_than_k(k, a):
    return sum([first_back_on_k(t, a) for t in range(1, k+1)])

print(f'Bероятность возвращения в состояние 11 не позднее чем за 6 шагов: \
{back_no_later_than_k(6, 10)}')

Bероятность возвращения в состояние 11 не позднее чем за 6 шагов: 0.243783987356


In [10]:
# 8. Cреднее время возвращения в состояние 10

def avg_time_back_to_a(a):
    return sum([t * first_back_on_k(t, a) for t in range(1, 100)])

print(f'Cреднее время возвращения в состояние 10: {avg_time_back_to_a(9)}')

Cреднее время возвращения в состояние 10: 10.367803375972475


In [13]:
# 9. Установившиеся вероятности
import copy

# Матрица

M_ = copy.deepcopy(P)
M_ = np.transpose(M_)
size = len(P)

for i in range(size):
    M_[i][i] = -P[i].sum()

# Расчет

B = np.zeros(size)
B[-1] = 1
M_[-1] = 1


M_1 = np.linalg.inv(M_)

X = M_1@B

print('Установившиеся вероятности:')
for i, item in enumerate(X):
    print(f'{i:>2}: {item}')

Установившиеся вероятности:
 0: 0.053460872340597286
 1: 0.04254506141589115
 2: 0.1446228877147753
 3: 0.030358890891404717
 4: 0.02352278382986286
 5: 0.03414917870070567
 6: 0.1645202846020898
 7: 0.18077045874013004
 8: 0.021449893087286065
 9: 0.02274328605648393
10: 0.2818564026207731
